In [ ]:
!pip install gradio
!pip install anthropic
!pip install langgraph langchain langchain_core

In [1]:
import gradio as gr
import pandas as pd
import tempfile
import os
from anthropic import Anthropic
from openai import OpenAI
import re
import traceback
import io
from contextlib import redirect_stdout
import numpy as np
import logging
import json
from typing import List, Optional, Dict, Any, Union
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import StateGraph, END

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('rent_roll_analyzer')

# Global variables and API keys


In [3]:
from typing import TypedDict, List, Optional, Union, Dict, Any

# Define the state as a TypedDict
class AgentState(TypedDict, total=False):
    messages: List[Dict[str, str]]
    df: Optional[pd.DataFrame]
    issues: List[str]
    execution_plan: Optional[str]
    needs_clarification: bool
    clarification_question: Optional[str]
    generate_code: bool
    code_execution_results: Optional[str]
    final_response: Optional[str]
    anthropic_client: Optional[Any]  # For Claude API
    openai_client: Optional[Any]     # For OpenAI API


In [4]:
def read_rent_roll_simple(file_path):
    """
    Improved function to read rent roll Excel files that handles special formatting
    commonly found in commercial real estate rent roll sheets.
    """
    # Read the raw Excel file with no header
    df = pd.read_excel(file_path, header=None)

    # Find the row containing the column headers
    header_row = None
    for i, row in df.iterrows():
        if row.iloc[0] == "Current":
            header_row = i + 1  # Headers are in the row after "Current"
            break

    if header_row is None:
        logger.warning("Could not find header row with 'Current' marker. Falling back to standard loading.")
        return pd.read_excel(file_path)

    # Get the headers
    headers = []
    for val in df.iloc[header_row]:
        if pd.isna(val):
            headers.append("NaN")  # Use "NaN" for empty header cells
        else:
            headers.append(str(val))

    # Create a new dataframe starting after the header row
    data_rows = df.iloc[(header_row+1):].values

    # Create a new dataframe with the extracted headers
    result_df = pd.DataFrame(data_rows, columns=headers)

    logger.info(f"Successfully loaded rent roll with {len(result_df)} rows using specialized loader")
    return result_df

In [5]:
def analyze_rent_roll_gpt(file_path, api_key):
    """
    Analyzes a CRE rent roll Excel file by sending the data rows to GPT-4.
    """
    # Load the rent roll
    try:
        df = read_rent_roll_simple(file_path)
        logger.info("File loaded successfully for GPT analysis.")
    except Exception as e:
        logger.error(f"Error loading file: {e}")
        return []

    # Initialize OpenAI client
    client = OpenAI(api_key=api_key)

    # Convert the DataFrame to CSV string format
    csv_data = df.to_csv(index=False)
    logger.info(f"Converted DataFrame to CSV with {len(df)} rows and {len(df.columns)} columns")

    # Enhance the system prompt to focus on general rent roll issues
    system_prompt = """
    You are a Commercial Real Estate rent roll expert specializing in identifying data quality, formatting, and consistency issues.

    When analyzing any CRE rent roll, rigorously check for these common categories of issues:

    1. DUPLICATE OR REDUNDANT ENTRIES: Look for any repeated charges, fees, or line items
    2. INCONSISTENT TERMINOLOGY: Identify any unclear, non-standard, or ambiguous descriptions
    3. DATE ANOMALIES: Flag any suspicious or illogical date patterns across move-in, lease start/end
    4. RENT DISCREPANCIES: Identify deviations between market rent values and actual charged amounts
    5. CALCULATION INCONSISTENCIES: Check if component charges properly sum to totals
    6. EXCEL ARTIFACTS: Identify any visible formulas, function calls, or spreadsheet mechanics
    7. FORMATTING IRREGULARITIES: Notice inconsistent data entry patterns or splitting of information
    8. BALANCE ANOMALIES: Identify unusual balances, especially negative values
    9. OCCUPANCY MISMATCHES: Look for occupied units with zero rent or vacant units with charges
    10. UNIT IDENTIFICATION PATTERNS: Check for inconsistencies in unit numbering or identification

    Be extremely thorough and specific in your analysis. Report ALL issues you find, regardless of how minor they may seem.
    DO NOT return "No issues detected" unless you've comprehensively analyzed the data for each category above.
    """

    # Use a simplified prompt focused on analyzing the raw CSV data
    prompt = (
        f"Please analyze this Commercial Real Estate rent roll data in CSV format and identify ALL potential issues "
        f"that could affect data quality, accuracy, or decision-making.\n\n{csv_data}\n\n"

        f"Based on your expertise in CRE rent rolls, provide a numbered list of ALL issues you can identify, including but not limited to:\n\n"

        f"- Any duplicate or redundant charges\n"
        f"- Unclear, non-standard, or inconsistent descriptions\n"
        f"- Suspicious or illogical date patterns\n"
        f"- Inconsistencies between market rent and actual rent values\n"
        f"- Calculation errors where components don't match totals\n"
        f"- Spreadsheet artifacts like visible formulas\n"
        f"- Inconsistent data entry patterns\n"
        f"- Unusual balance values\n"
        f"- Occupancy status mismatches\n"
        f"- Inconsistent unit numbering or identification\n\n"

        f"IMPORTANT: For each issue found, please reference the specific unit(s) affected and explain why it's problematic. "
        f"Be comprehensive - rent roll accuracy is critical for CRE investment and property management decisions."
    )

    try:
        logger.info("Sending request to GPT-4 for analysis...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000,
            temperature=0.3
        )
        response_text = response.choices[0].message.content
        logger.info("Received response from GPT-4.")

        # Simple parsing of the response - split by numbered items
        lines = response_text.split('\n')
        issues = []
        current_issue = ""

        for line in lines:
            # If it's a new numbered item
            if line.strip() and line[0].isdigit() and '. ' in line[:5]:
                # If we were building a previous issue, add it
                if current_issue:
                    issues.append(current_issue.strip())
                current_issue = line.strip()
            elif line.strip() and current_issue:
                # Continue building the current issue
                current_issue += " " + line.strip()

        # Add the last issue if there is one
        if current_issue:
            issues.append(current_issue.strip())

        if not issues:
            issues.append("No issues detected by GPT-4.")

        logger.info(f"Identified {len(issues)} issues in the rent roll")
        return issues

    except Exception as e:
        logger.error(f"Error calling GPT-4 for analysis: {e}")
        logger.error(traceback.format_exc())
        return ["Failed to analyze rent roll due to API error."]

In [6]:
def determine_action(state):
    """Decide whether to answer directly, ask for clarification, or generate code."""

    messages = state["messages"]
    user_message = messages[-1]["content"] if messages[-1]["role"] == "user" else ""
    df = state["df"]

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    # Get column information for context
    if df is not None:
        try:
            # Safer way to get column data types
            column_info = []
            for col in df.columns:
                try:
                    dtype_str = str(df[col].dtype)  # Convert dtype to string directly
                    column_info.append(f"- {col}: {dtype_str}")
                except:
                    column_info.append(f"- {col}: unknown type")
            column_info_str = "\n".join(column_info)
            df_preview = df.head(3).to_string()
        except Exception as e:
            logger.error(f"Error getting column info: {e}")
            column_info_str = "Error retrieving column information"
            df_preview = "Error retrieving data preview"
    else:
        column_info_str = "No dataframe loaded"
        df_preview = "No data available"

    # Use GPT-4 to analyze the query and determine the best action
    prompt = f"""
    User query: {user_message}

    Dataframe information:
    - Rows: {len(df) if df is not None else 'No data loaded'}
    - Columns: {column_info_str}

    Data preview:
    {df_preview}

    Analyze the user query and determine the most appropriate action:
    1. If the query is ambiguous or lacks specificity, choose "ask_clarification"
    2. If the query can be answered with a simple explanation without analysis, choose "text_response"
    3. If the query requires data analysis, calculations, or visualizations, choose "generate_code"

    Respond with a JSON object containing:
    {{"action": "ask_clarification" | "text_response" | "generate_code", "reason": "brief explanation"}}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a decision-making agent for a rent roll analysis system. Output ONLY a JSON object with the determined action and reason."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.2
        )

        response_text = response.choices[0].message.content

        # Extract JSON from the response
        json_match = re.search(r'{.*}', response_text, re.DOTALL)
        if json_match:
            action_data = json.loads(json_match.group(0))
            action = action_data.get("action", "text_response")
        else:
            # Default to text response if parsing fails
            action = "text_response"

        logger.info(f"Determined action using GPT-4: {action}")

        # Create a new state dict with updated values
        new_state = dict(state)  # Create a copy
        new_state["needs_clarification"] = action == "ask_clarification"
        new_state["generate_code"] = action == "generate_code"

        return new_state
    except Exception as e:
        logger.error(f"Error in determine_action: {e}")
        # Default to text response on error
        new_state = dict(state)
        new_state["needs_clarification"] = False
        new_state["generate_code"] = False
        return new_state

In [7]:

def ask_clarification(state: AgentState) -> Dict:
    """Generate a clarification question for the user using GPT-4."""

    messages = state["messages"]
    user_message = messages[-1]["content"] if messages[-1]["role"] == "user" else ""

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": """You are a commercial real estate rent roll analyst.
                Generate a clear, specific clarification question to better understand
                what the user is asking about their rent roll data."""},
                {"role": "user", "content": f"My question is: {user_message}"}
            ],
            max_tokens=300,
            temperature=0.3
        )

        clarification_question = response.choices[0].message.content

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["clarification_question"] = clarification_question
        new_state["final_response"] = clarification_question

        # Add the clarification question to the messages
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": clarification_question})
        new_state["messages"] = new_messages

        logger.info(f"Generated clarification question using GPT-4: {clarification_question[:50]}...")
        return new_state
    except Exception as e:
        logger.error(f"Error in ask_clarification: {e}")
        # Fallback to a generic clarification question
        generic_question = "Could you please clarify what specific aspect of the rent roll you'd like me to analyze?"

        new_state = dict(state)
        new_state["clarification_question"] = generic_question
        new_state["final_response"] = generic_question

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": generic_question})
        new_state["messages"] = new_messages

        return new_state

In [8]:
def generate_text_response(state):
    """Generate a simple text response to the user query using GPT-4."""

    messages = state["messages"]
    df = state["df"]
    issues = state["issues"]

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    # Prepare context for GPT-4
    issues_text = "\n".join([f"- {issue}" for issue in issues])

    # Get column and data preview for context
    if df is not None:
        column_info = ", ".join(df.columns)
        data_stats = []
        for col in df.columns[:10]:  # Limit to first 10 columns to avoid token limits
            try:
                if pd.api.types.is_numeric_dtype(df[col]):
                    stat = f"- {col}: min={df[col].min()}, max={df[col].max()}, mean={df[col].mean():.2f}, null={df[col].isna().sum()}"
                else:
                    unique_vals = df[col].nunique()
                    stat = f"- {col}: unique values={unique_vals}, null={df[col].isna().sum()}"
                data_stats.append(stat)
            except:
                data_stats.append(f"- {col}: [error calculating stats]")
        data_stats_str = "\n".join(data_stats)
        df_preview = df.head(3).to_string()
    else:
        column_info = "No columns available"
        data_stats_str = "No data statistics available"
        df_preview = "No data preview available"

    system_prompt = f"""You are a commercial real estate rent roll analyst.
    The rent roll data has {len(df) if df is not None else 0} rows and
    {len(df.columns) if df is not None else 0} columns.

    Column information: {column_info}

    Data statistics:
    {data_stats_str}

    Data preview:
    {df_preview}

    Identified issues:
    {issues_text}

    Provide a concise, informative answer to the user's question.
    Focus on being helpful and direct, with only 1-2 paragraphs.
    Do not include code or detailed analysis unless absolutely necessary.
    """

    # Extract system message and filter other messages
    filtered_messages = []
    for msg in messages:
        if msg["role"] != "system":
            filtered_messages.append(msg)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                *filtered_messages
            ],
            max_tokens=1000,
            temperature=0.3
        )

        text_response = response.choices[0].message.content

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["final_response"] = text_response

        # Add the response to the messages
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": text_response})
        new_state["messages"] = new_messages

        logger.info(f"Generated text response using GPT-4: {text_response[:50]}...")
        return new_state
    except Exception as e:
        logger.error(f"Error in generate_text_response: {e}")
        # Fallback to a generic response
        fallback_response = "I'm sorry, I'm having trouble analyzing your rent roll data right now. Could you try rephrasing your question?"

        new_state = dict(state)
        new_state["final_response"] = fallback_response

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": fallback_response})
        new_state["messages"] = new_messages

        return new_state

In [9]:
def trim_dataframe_output(output_text, max_rows=20, max_chars=None):
    """
    Extremely simplified function that just returns the first 20 lines of output.

    Args:
        output_text: The text output
        max_rows: Maximum number of rows to keep (default: 20)
        max_chars: Not used, kept for compatibility

    Returns:
        Trimmed text showing only top rows
    """
    lines = output_text.split('\n')

    if len(lines) <= max_rows:
        return output_text

    trimmed_lines = lines[:max_rows]
    trimmed_lines.append(f"... [output truncated, showing first {max_rows} lines only] ...")

    return '\n'.join(trimmed_lines)

In [10]:

from datetime import datetime
def save_dataframe_version(df, operation_description=""):
    """Save the current state of the dataframe as both CSV and Excel files.

    Args:
        df: The dataframe to save
        operation_description: A string describing what operation was performed

    Returns:
        version_name: The name of the version that was saved
    """
    import os
    from datetime import datetime

    # Create versions directory if it doesn't exist
    versions_dir = "rent_roll_versions"
    os.makedirs(versions_dir, exist_ok=True)

    # Generate version name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    version_name = f"v_{timestamp}"

    # Create filenames for both CSV and Excel
    csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
    excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

    # Save as CSV
    df.to_csv(csv_filename, index=False)

    # Save as Excel
    df.to_excel(excel_filename, index=False, engine='openpyxl')

    # Add version metadata to the registry
    if 'app_state' in globals():
        version_info = {
            'name': version_name,
            'description': operation_description,
            'timestamp': timestamp,
            'filename': csv_filename,  # Keep CSV as primary for backward compatibility
            'excel_filename': excel_filename,  # Add Excel filename
            'is_original': len(app_state["df_versions"]) == 0  # First one is original
        }
        app_state["df_versions"].append(version_info)

    print(f"✓ Saved dataframe version {version_name}: {operation_description}")
    print(f"  - CSV: {csv_filename}")
    print(f"  - Excel: {excel_filename}")

    # Return the version name for reference
    return version_name

def get_versions_info_for_prompt():
    """Generate version information for the Claude prompt."""
    if not app_state["df_versions"]:
        return "No versions available yet."

    # Find the original version
    original = next((v for v in app_state["df_versions"] if v.get('is_original')), app_state["df_versions"][0])

    # Get the latest version
    latest = app_state["df_versions"][-1]

    # Format all versions
    all_versions = []
    for i, version in enumerate(app_state["df_versions"]):
        status = []
        if version == original:
            status.append("ORIGINAL")
        if version == latest:
            status.append("LATEST")

        status_str = f" ({', '.join(status)})" if status else ""
        all_versions.append(f"{i+1}. {version['name']}{status_str}: {version['description']}")

    versions_text = "\n".join(all_versions)

    return f"""
DATAFRAME VERSION HISTORY:
{versions_text}

Original version: {original['name']}
Latest version: {latest['name']}
Total versions: {len(app_state["df_versions"])}
"""

def generate_code_and_execute(state: AgentState) -> Dict:
    """
    Generate and execute code using a two-step AI approach:
    1. Use GPT-4 to create an optimal prompt for Claude
    2. Have Claude generate the code based on this optimized prompt
    3. Execute the code and handle errors with up to 3 retries
    """
    messages = state["messages"]
    df = state["df"]

    # Get OpenAI client from state or create new one
    openai_client = state.get("openai_client") or OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
    # Get Anthropic client from state or create new one
    anthropic_client = state.get("anthropic_client") or Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

    # Get column information for context
    column_info = ", ".join(df.columns) if df is not None else "No columns available"

    # Create a snapshot of the dataframe to show Claude
    df_sample = df.head(10).to_string() if df is not None else "No data available"

    # Create versions directory if it doesn't exist
    versions_dir = "rent_roll_versions"
    os.makedirs(versions_dir, exist_ok=True)

    # Print initial state for debugging
    print(f"\n==== STARTING CODE GENERATION ====")
    print(f"User query: {messages[-1]['content'] if messages[-1]['role'] == 'user' else 'No user query found'}")
    print(f"Dataframe has {len(df) if df is not None else 0} rows and {len(df.columns) if df is not None else 0} columns")

    try:
        # First, use GPT-4 to create the optimal prompt for Claude
        print("\n==== STEP 1: GENERATING PROMPT WITH GPT-4 ====")
        versions_info = get_versions_info_for_prompt()
        # System prompt for GPT-4 to create a Claude prompt
        gpt_system_prompt = f"""You are an expert at creating prompts for Claude AI to generate code.
        Your task is to analyze the user query history and convert it into an optimal prompt for Claude to generate Python code that analyzes a rent roll dataframe.

        CRITICAL INFORMATION: The dataframe is ALREADY LOADED and available as 'df'.
        It contains REAL DATA with {len(df)} rows and {len(df.columns)} columns.

        Here is a sample of the actual data (first 5 rows):
        {df_sample}

        The dataframe has the following columns: {column_info}

        # IMPORTANT: DATAFRAME VERSION MANAGEMENT
        {versions_info}

        # IMPORTANT VERSION IDENTIFICATION:
        - Versions are stored in chronological order by timestamp
        - The original version is always the first one saved (earliest timestamp)
        - The latest version is always the most recent one saved (latest timestamp)
        - When a user says "original dataframe," load the version with the earliest timestamp
        - When a user says "latest version," use the current df (which is already the latest)
        - When a user specifies a version by name (e.g., "v_20250518_112345"), load that exact version

        ALL versions are saved as CSV files in the "rent_roll_versions" directory.
        For example, to load a specific version:

        ```python
        # To load a specific version (e.g., the original version)
        import pandas as pd
        import os

        # Example: Load the original version
        original_version_name = "{{app_state["df_versions"][0]['name'] if app_state["df_versions"] else "v_example"}}"
        original_file_path = os.path.join("rent_roll_versions", f"rent_roll_{{original_version_name}}.csv")
        original_df = pd.read_csv(original_file_path)

        print(f"Loaded original version: {{original_version_name}}")
        print(f"Shape: {{original_df.shape}}")

        # You can either work with this as a separate dataframe, or replace the current df:
        # df = original_df  # This would replace the current df with the original
        ```

        If you make any changes to the dataframe, ALWAYS save a new version using save_dataframe_version().

        Some important guidelines to include in your prompt to Claude:
        1. The variable 'df' is ALREADY DEFINED and CONTAINS DATA. Claude must not say "I need to see the data first"
        2. Claude should explain its approach step by step before showing code
        3. Code must be wrapped in ```python and ``` blocks
        4. Code MUST display ALL rows in the output when showing tables (no limiting rows)
        5. Claude should not attempt to clean data unless specifically requested
        6. Code should include proper error handling
        7. IMPORTANT: After performing any analysis or showing results, Claude should ALWAYS call the save_dataframe_version() function to maintain version history, even if no changes were made to the dataframe.
        8. CRITICAL: Claude should NOT use try-except blocks in its code. Any errors should be allowed to propagate naturally. This ensures that our retry system can properly handle errors.

        Your output will be directly sent to Claude, so format it as a complete system prompt.
        Include any table formatting functions that might be useful.

        Make sure to include these helper functions in your prompt:

        ```python
        # For tabular display with proper formatting (PREFERRED METHOD):
        def print_formatted_table(df, title=None): #Print a dataframe with proper formatting without modifying data
            if title:
                print(f"\\n{{title}}")
                print("=" * 80)

            # Create a display copy (doesn't change original df)
            display_df = df.copy()

            # Set pandas display options for better readability
            # Show ALL rows - no limits
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            pd.set_option('display.colheader_justify', 'left')
            pd.set_option('display.precision', 2)

            # Display the dataframe - ALL rows will be shown
            print(display_df)

            # Reset display options to default
            pd.reset_option('display.max_rows')
            pd.reset_option('display.max_columns')
            pd.reset_option('display.width')
            pd.reset_option('display.colheader_justify')
            pd.reset_option('display.precision')
        ```

        ```python
        # For bordered table display with precise control:
        def print_bordered_table(df, title=None): #Print a dataframe with borders for better readability - SHOWS ALL ROWS
            if title:
                print(f"\\n{{title}}")
                print("=" * 80)

            if len(df) == 0:
                print("No data available")
                return

            # Create a display copy (doesn't change original data)
            display_df = df.copy()

            # Calculate column widths for display purposes only
            col_widths = {{}}
            for col in display_df.columns:
                # Convert values to string only for width calculation
                col_values = display_df[col].astype(str)
                max_data_width = col_values.str.len().max()
                col_widths[col] = max(len(str(col)), max_data_width) + 2  # +2 for padding

            # Create header row
            header = "| " + " | ".join(str(col).ljust(col_widths[col]) for col in display_df.columns) + " |"
            separator = "+" + "+".join("-" * (col_widths[col] + 2) for col in display_df.columns) + "+"

            # Print header
            print(separator)
            print(header)
            print(separator)

            # Print ALL rows - NO LIMIT
            for i in range(len(display_df)):
                row = display_df.iloc[i]
                row_str = "| " + " | ".join(str(val).ljust(col_widths[col]) for col, val in row.items()) + " |"
                print(row_str)

            print(separator)
            print(f"Total rows: {{len(display_df)}}")
        ```

        ```python
        # Function to save dataframe versions
        def save_dataframe_version(df, operation_description=""):
            \"\"\"Save the current state of the dataframe as both CSV and Excel files.

            This function should be called whenever you make changes to the dataframe,
            or after generating analysis results, to maintain version history.

            Args:
                df: The dataframe to save
                operation_description: A string describing what operation was performed

            Returns:
                version_name: The name of the version that was saved
            \"\"\"
            import os
            from datetime import datetime

            # Create versions directory if it doesn't exist
            versions_dir = "rent_roll_versions"
            os.makedirs(versions_dir, exist_ok=True)

            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{{timestamp}}"

            # Create filenames for both CSV and Excel
            csv_filename = os.path.join(versions_dir, f"rent_roll_{{version_name}}.csv")
            excel_filename = os.path.join(versions_dir, f"rent_roll_{{version_name}}.xlsx")

            # Save as CSV
            df.to_csv(csv_filename, index=False)

            # Save as Excel
            df.to_excel(excel_filename, index=False, engine='openpyxl')

            print(f"✓ Saved dataframe version {{version_name}}: {{operation_description}}")
            print(f"  - CSV: {{csv_filename}}")
            print(f"  - Excel: {{excel_filename}}")

            # Return the version name for reference
            return version_name
        ```
        """
        # Filter out system messages and DON'T trim dataframe outputs in the conversation history
        filtered_messages = []
        for msg in messages:
            if msg["role"] != "system":
                # Don't trim here anymore
                filtered_messages.append({"role": msg["role"], "content": msg["content"]})

        # Convert the messages to the format expected by OpenAI
        gpt_messages = [{"role": "system", "content": gpt_system_prompt}]
        for msg in filtered_messages:
            gpt_messages.append(msg)

        # Add a final message explaining the task clearly
        gpt_messages.append({
            "role": "user",
            "content": "Based on this conversation history, create the optimal Claude prompt to generate Python code for rent roll analysis. The prompt should emphasize that the dataframe already exists and is loaded as 'df', that ALL rows should be displayed when requested, and that versions should be saved with save_dataframe_version() function."
        })

        # Get the optimized prompt from GPT-4
        gpt_response = openai_client.chat.completions.create(
            model="gpt-4.1",
            messages=gpt_messages,
            max_tokens=3000,
            temperature=0.3
        )

        claude_system_prompt = gpt_response.choices[0].message.content

        # Print the generated prompt for debugging
        print("\n==== GPT-4 GENERATED PROMPT FOR CLAUDE ====")
        print(claude_system_prompt[:500] + "..." if len(claude_system_prompt) > 500 else claude_system_prompt)
        print("==== END OF PROMPT (TRUNCATED) ====\n")

        logger.info("Generated optimized prompt for Claude using GPT-4")

        # Now use the GPT-4 generated prompt to ask Claude for code
        print("\n==== STEP 2: SENDING TO CLAUDE FOR CODE GENERATION ====")
        logger.info("Sending optimized prompt to Claude for code generation")

        # Prepare messages for Claude with a dataframe sample
        claude_messages = filtered_messages.copy()

        # Add a sample of the actual dataframe to help Claude understand the data exists
        sample_message = {
            "role": "user",
            "content": f"Here is a sample of the dataframe that's already loaded as 'df':\n{df_sample}\n\nPlease process my request using this data and remember to save versions with save_dataframe_version()."
        }
        claude_messages.append(sample_message)

        # Try to get code from Claude
        claude_response = anthropic_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            system=claude_system_prompt,
            messages=claude_messages,
            max_tokens=3500,
            temperature=0.3
        )

        # Extract the response text from Claude
        response_text = claude_response.content[0].text

        # Print Claude's response for debugging
        print("\n==== CLAUDE'S RESPONSE ====")
        print(response_text[:500] + "..." if len(response_text) > 500 else response_text)
        print("==== END OF CLAUDE RESPONSE (TRUNCATED) ====\n")

        # Extract code blocks
        code_blocks = re.findall(r'```python\s*(.*?)\s*```', response_text, re.DOTALL)

        # Print extracted code blocks for debugging
        print(f"\n==== EXTRACTED {len(code_blocks)} CODE BLOCKS ====")
        for i, block in enumerate(code_blocks):
            print(f"\n-- Code Block {i+1} --")
            print(block[:200] + "..." if len(block) > 200 else block)

        # If no code blocks are found, add emergency code
        if len(code_blocks) == 0:
            emergency_code = """
            # Emergency code to display the dataframe
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)

            print("\\n=== RENT ROLL DATA ===\\n")
            print(f"Displaying all {len(df)} rows and {len(df.columns)} columns\\n")

            # Print the entire dataframe
            print(df)

            # Save a version of the dataframe
            from datetime import datetime
            import os

            # Create versions directory if it doesn't exist
            versions_dir = "rent_roll_versions"
            os.makedirs(versions_dir, exist_ok=True)

            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{timestamp}"

            # Create filename
            filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")

            # Save dataframe
            df.to_csv(filename, index=False)

            print(f"✓ Saved dataframe version {version_name}: Emergency display of data")
            """
            code_blocks.append(emergency_code)
            print("\n-- Added Emergency Code Block --")
            print("Emergency code added since Claude didn't generate code")

        # Define helper functions
        def print_formatted_table(df, title=None):
            if title:
                print(f"\n{title}")
                print("=" * 80)

            # Create a display copy (doesn't change original df)
            display_df = df.copy()

            # Set pandas display options for better readability
            # Show ALL rows - no limits
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            pd.set_option('display.colheader_justify', 'left')
            pd.set_option('display.precision', 2)

            # Display the dataframe - ALL rows will be shown
            print(display_df)

            # Reset display options to default
            pd.reset_option('display.max_rows')
            pd.reset_option('display.max_columns')
            pd.reset_option('display.width')
            pd.reset_option('display.colheader_justify')
            pd.reset_option('display.precision')

        def print_bordered_table(df, title=None):
            if title:
                print(f"\n{title}")
                print("=" * 80)

            if len(df) == 0:
                print("No data available")
                return

            # Create a display copy (doesn't change original data)
            display_df = df.copy()

            # Calculate column widths for display purposes only
            col_widths = {}
            for col in display_df.columns:
                # Convert values to string only for width calculation
                col_values = display_df[col].astype(str)
                max_data_width = col_values.str.len().max()
                col_widths[col] = max(len(str(col)), max_data_width) + 2  # +2 for padding

            # Create header row
            header = "| " + " | ".join(str(col).ljust(col_widths[col]) for col in display_df.columns) + " |"
            separator = "+" + "+".join("-" * (col_widths[col] + 2) for col in display_df.columns) + "+"

            # Print header
            print(separator)
            print(header)
            print(separator)

            # Print ALL rows - NO LIMIT
            for i in range(len(display_df)):
                row = display_df.iloc[i]
                row_str = "| " + " | ".join(str(val).ljust(col_widths[col]) for col, val in row.items()) + " |"
                print(row_str)

            print(separator)
            print(f"Total rows: {len(display_df)}")

        # Add to globals_dict before executing code
        globals_dict = {
            "df": df,
            "pd": pd,
            "np": np,
            "os": os,                   # Add os for folder creation
            "datetime": datetime,       # Add datetime for timestamp
            "versions_dir": versions_dir,  # Pass the versions directory
            "print_formatted_table": print_formatted_table,  # Add the helper function
            "print_bordered_table": print_bordered_table,    # Add the helper function
            "save_dataframe_version": save_dataframe_version  # Make sure this is defined too
        }

        execution_results = ""
        all_executed_successfully = False
        max_retries = 5  # Maximum number of retries
        retry_count = 0  # Initialize retry counter
        failed_code = ""  # Store the failed code for context
        error_msg = ""    # Store the error message

        print("\n==== STEP 3: EXECUTING CODE WITH RETRIES ====")

        # Main retry loop
        while not all_executed_successfully and retry_count <= max_retries:
            # If this is a retry attempt (not the first try)
            if retry_count > 0:
                print(f"\n==== RETRY ATTEMPT {retry_count}/{max_retries} ====")

                # Create a retry message with more details each time
                retry_message = {
                    "role": "user",
                    "content": f"""The code you provided failed with this error: {error_msg}

                    Here is the code that failed:
                    ```python
                    {failed_code}
                    ```

                    This is retry attempt {retry_count} of {max_retries}.

                    {"After multiple attempts, please try a completely different approach." if retry_count >= 2 else "Please fix this specific error."}
                    IMPORTANT: DO NOT use try-except blocks in your code. Allow any errors to propagate naturally so our system can detect them.
                    Please fix this code to handle the specific error while maintaining the requirement to show ALL rows in the output and saving a version with save_dataframe_version().
                    Return the corrected code wrapped in ```python and ``` blocks."""
                }

                # Add this feedback to the messages
                fix_messages = claude_messages.copy()
                fix_messages.append({"role": "assistant", "content": response_text})
                fix_messages.append(retry_message)

                # Get Claude's fixed code
                retry_response = anthropic_client.messages.create(
                    model="claude-3-7-sonnet-20250219",
                    system=claude_system_prompt,
                    messages=fix_messages,
                    max_tokens=3500,
                    temperature=0.3
                )

                retry_text = retry_response.content[0].text
                print(f"\n==== CLAUDE'S FIX SUGGESTION (ATTEMPT {retry_count}) ====")
                print(retry_text[:500] + "..." if len(retry_text) > 500 else retry_text)

                # Extract the fixed code blocks
                fixed_code_blocks = re.findall(r'```python\s*(.*?)\s*```', retry_text, re.DOTALL)

                if fixed_code_blocks:
                    # Use the first fixed code block
                    code_to_execute = fixed_code_blocks[0]

                    # Update response text to include the fix explanation
                    fix_explanation = f"\n\n**🔧 Code Fix (Attempt {retry_count}):**\n"
                    fix_explanation += f"The code encountered an error. Here's the fix for retry attempt {retry_count}:\n"
                    fix_explanation += "\n```python\n" + code_to_execute + "\n```\n"

                    if retry_count == 1:
                        # First retry - add to original response
                        response_text = response_text + fix_explanation
                    else:
                        # Subsequent retries - replace previous fix explanation
                        prev_fix_marker = f"**🔧 Code Fix (Attempt {retry_count-1}):**"
                        if prev_fix_marker in response_text:
                            # Replace previous fix with new one
                            response_text = response_text.replace(
                                prev_fix_marker,
                                f"**🔧 Code Fix (Attempt {retry_count}):**"
                            )
                        else:
                            # Just append this fix
                            response_text = response_text + fix_explanation
                else:
                    # If no code blocks found in retry, try emergency code
                    code_to_execute = f"""
                    # Emergency code for retry {retry_count}
                    print(f"\\n=== EMERGENCY DISPLAY (RETRY {retry_count}) ===\\n")
                    print(f"DataFrame shape: {{df.shape}}")
                    print("\\nColumn names:")
                    for col in df.columns:
                        print(f"- {{col}}")

                    print("\\nFirst 10 rows:")
                    print(df.head(10))

                    save_dataframe_version(df, f"Emergency display after retry {retry_count}")
                    """
                    print(f"No code blocks found in retry. Using emergency code.")
            else:
                # Initial execution (not a retry)
                # Run the original code block
                if code_blocks:
                    code_to_execute = code_blocks[0]  # Use the first code block
                else:
                    # This should not happen due to the earlier check, but just in case
                    code_to_execute = """
                    print("No code blocks found. Displaying basic dataframe info.")
                    print(f"DataFrame shape: {df.shape}")
                    print(df.head())
                    save_dataframe_version(df, "Automatic save after initial execution")
                    """

            # Execute the current code
            print(f"\n{'Executing' if retry_count == 0 else 'Retrying'} code...")
            output_buffer = io.StringIO()
            try:
                # Store the code in case it fails
                failed_code = code_to_execute

                with redirect_stdout(output_buffer):
                    exec(code_to_execute, globals_dict)

                execution_output = output_buffer.getvalue()
                print(f"Execution {'successful' if retry_count == 0 else 'fixed on retry ' + str(retry_count)}! Output length: {len(execution_output)} characters")
                print(execution_output[:200] + "..." if len(execution_output) > 200 else execution_output)

                # ONLY trim the execution output for storing, not the entire response
                trimmed_output = trim_dataframe_output(execution_output, max_rows=20)

                # Format the results message based on retry count
                if retry_count == 0:
                    results_msg = "**✅ Code Execution Results:**"
                else:
                    results_msg = f"**✅ Code Execution Results (After Fix Attempt {retry_count}):**"

                execution_results = f"\n\n{results_msg}\n```\n{trimmed_output}\n```\n"

                # Check if a version was saved
                if "✓ Saved dataframe version" not in execution_output:
                    # Auto-save a version
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    version_name = f"v_{timestamp}"
                    csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
                    excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

                    # Save both CSV and Excel
                    df.to_csv(csv_filename, index=False)
                    df.to_excel(excel_filename, index=False, engine='openpyxl')

                    save_message = f"✓ Saved dataframe version {version_name}: Automatic save after {'execution' if retry_count == 0 else 'retry ' + str(retry_count)}"
                    print(save_message)
                    print(f"  - CSV: {csv_filename}")
                    print(f"  - Excel: {excel_filename}")
                    execution_results += f"\n{save_message}\n"

                # Mark as successful and break the retry loop
                all_executed_successfully = True
                logger.info(f"Successfully executed code {'' if retry_count == 0 else 'on retry ' + str(retry_count)}")
                break

            except Exception as e:
                # Execution failed
                error_msg = f"Error: {str(e)}"
                print(f"Execution failed with error: {error_msg}")

                # Log the error
                if retry_count == 0:
                    execution_results = f"\n\n**❌ Code Execution Failed:**\n```\n{error_msg}\n```\n"
                else:
                    execution_results = f"\n\n**❌ Code Execution Failed (Retry {retry_count}):**\n```\n{error_msg}\n```\n"

                logger.error(f"Code execution failed on {'initial attempt' if retry_count == 0 else 'retry ' + str(retry_count)}: {e}")
                logger.error(traceback.format_exc())

                # Increment retry counter
                retry_count += 1

                # If we've hit max retries and still failed, try emergency display as last resort
                if retry_count > max_retries:
                    print("\n==== MAX RETRIES REACHED, TRYING EMERGENCY DISPLAY ====")

                    # Create emergency display code
                    emergency_code = """
                    try:
                        print("\\n=== EMERGENCY FALLBACK DISPLAY ===\\n")
                        print(f"DataFrame shape: {df.shape}")
                        print("\\nColumn names:")
                        for col in df.columns:
                            print(f"- {col}")

                        print("\\nFirst 10 rows:")
                        print(df.head(10))

                        # Try to show some basic stats about numeric columns
                        try:
                            numeric_cols = df.select_dtypes(include=['number']).columns
                            if len(numeric_cols) > 0:
                                print("\\nBasic statistics for numeric columns:")
                                print(df[numeric_cols].describe())
                        except Exception as stats_err:
                            print(f"Could not generate statistics: {stats_err}")

                        # Save version - both CSV and Excel
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        version_name = f"v_{timestamp}_emergency"
                        csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
                        excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

                        df.to_csv(csv_filename, index=False)
                        df.to_excel(excel_filename, index=False, engine='openpyxl')

                        print(f"✓ Saved dataframe version {version_name}: Emergency display after all retries failed")
                        print(f"  - CSV: {csv_filename}")
                        print(f"  - Excel: {excel_filename}")
                    except Exception as e_inner:
                        print(f"Even emergency display failed: {e_inner}")
                    """


                    output_buffer = io.StringIO()
                    try:
                        with redirect_stdout(output_buffer):
                            exec(emergency_code, globals_dict)

                        emergency_output = output_buffer.getvalue()
                        # Only trim the emergency output, not the whole response
                        execution_results += f"\n\n**⚠️ Emergency Data Display (After {max_retries} Failed Retries):**\n```\n{trim_dataframe_output(emergency_output, max_rows=20)}\n```\n"
                    except Exception as e_final:
                        print(f"Emergency fallback also failed: {e_final}")
                        execution_results += f"\n\n**❌ All Recovery Attempts Failed**\n"

        # Add a note about the hybrid approach and retry attempts
        if retry_count > 0 and all_executed_successfully:
            hybrid_note = f"\n\n**📝 Note:** This analysis was performed using a hybrid approach with GPT-4 and Claude. The code was successfully fixed after {retry_count} retry attempts."
        elif retry_count > max_retries:
            hybrid_note = f"\n\n**📝 Note:** This analysis was attempted using a hybrid approach with GPT-4 and Claude, but all {max_retries} retry attempts failed. Some basic information was displayed as a fallback."
        else:
            hybrid_note = "\n\n**📝 Note:** This analysis was performed using a hybrid approach: GPT-4 optimized the prompt, and Claude generated and executed the code for detailed rent roll analysis."

        # Combine the response and execution results
        full_response = response_text + execution_results + hybrid_note

        print("\n==== FINAL RESPONSE GENERATED ====")
        print(f"Original response length: {len(full_response)} characters")
        print(f"Retry attempts: {retry_count}")
        print(f"Execution successful: {all_executed_successfully}")

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["code_execution_results"] = execution_results
        new_state["final_response"] = full_response  # Don't trim the full response

        # Add the response to the messages - don't trim it here either
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": full_response})
        new_state["messages"] = new_messages

        logger.info("Code generation and execution complete using hybrid GPT-4/Claude approach")
        print("\n==== CODE GENERATION COMPLETE ====")

        return new_state

    except Exception as e:
        logger.error(f"Error in hybrid code generation: {e}")
        logger.error(traceback.format_exc())
        print(f"\n==== ERROR IN CODE GENERATION ====\n{e}\n{traceback.format_exc()}")

        # Try to save a version even on error
        try:
            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{timestamp}_system_error"

            # Create filenames
            csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
            excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

            # Save both formats
            df.to_csv(csv_filename, index=False)
            df.to_excel(excel_filename, index=False, engine='openpyxl')

            save_message = f"✓ Saved dataframe version {version_name}: System error - {str(e)[:100]}"
            print(save_message)
            print(f"  - CSV: {csv_filename}")
            print(f"  - Excel: {excel_filename}")
        except Exception as save_error:
            print(f"Failed to save error version: {save_error}")

        # Fallback to a generic response
        fallback_response = f"""
        I'm sorry, I encountered an issue while generating and executing code for your request.

        **Technical Details:** {str(e)}

        Could you try asking your question in a different way? For complex analyses, it sometimes helps to break down your request into smaller, more specific questions.
        """

        new_state = dict(state)
        new_state["final_response"] = fallback_response

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": fallback_response})
        new_state["messages"] = new_messages

        return new_state

In [11]:
# Build the LangGraph workflow
def create_agentic_rent_roll_analyzer():
    """Create and return the agentic rent roll analyzer workflow."""

    # Create the graph
    workflow = StateGraph(AgentState)

    # Add nodes to the graph
    workflow.add_node("determine_action", determine_action)
    workflow.add_node("ask_clarification", ask_clarification)
    workflow.add_node("generate_text_response", generate_text_response)
    workflow.add_node("generate_code_and_execute", generate_code_and_execute)

    # Set the entry point
    workflow.set_entry_point("determine_action")

    # Define conditional edges based on dictionary state values
    workflow.add_conditional_edges(
        "determine_action",
        lambda state: "ask_clarification" if state.get("needs_clarification") else
                      "generate_code_and_execute" if state.get("generate_code") else
                      "generate_text_response"
    )

    # Add edges to END
    workflow.add_edge("ask_clarification", END)
    workflow.add_edge("generate_text_response", END)
    workflow.add_edge("generate_code_and_execute", END)

    # Compile the graph
    agentic_analyzer = workflow.compile()

    return agentic_analyzer


In [12]:
def upload_rent_roll(file, anthropic_api_key, openai_api_key, auto_analyze):
    """Process the uploaded rent roll file and initialize the chat."""
    global app_state

    logger.info("Starting rent roll upload and processing")

    # Use the default API keys if none are provided
    anthropic_key = anthropic_api_key if anthropic_api_key else DEFAULT_ANTHROPIC_API_KEY
    openai_key = openai_api_key if openai_api_key else DEFAULT_OPENAI_API_KEY
    logger.info("API keys configured")

    # Validate inputs
    if not file:
        logger.warning("No file uploaded")
        return "Please upload a rent roll Excel file.", None, gr.update(visible=False)

    try:
        # Save the uploaded file to a temporary location
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx')
        temp_file.close()
        file_path = temp_file.name
        logger.info(f"Created temporary file: {file_path}")

        # Copy the uploaded file to our temporary location
        with open(file.name, 'rb') as src_file, open(file_path, 'wb') as dst_file:
            dst_file.write(src_file.read())
        logger.info("File copied to temporary location")

        # Use our improved rent roll loader
        try:
            logger.info("Loading rent roll with specialized loader...")
            rent_roll_df = read_rent_roll_simple(file_path)
        except Exception as e:
            logger.warning(f"Error with specialized loader: {e}. Falling back to standard loading.")
            # Fallback to basic loading
            rent_roll_df = pd.read_excel(file_path)
            logger.info("Fallback: Loaded rent roll with default pandas settings")

        logger.info(f"Loaded rent roll data: {len(rent_roll_df)} rows, {len(rent_roll_df.columns)} columns")

        # Auto-analyze with GPT if selected
        if auto_analyze:
            logger.info("Auto-analyze option selected. Calling GPT for analysis...")
            issues_list = analyze_rent_roll_gpt(file_path, openai_key)  # Use OpenAI key for this
            logger.info(f"GPT analysis complete. Found {len(issues_list)} issues.")
        else:
            # Create empty issues list if not auto-analyzing
            issues_list = []
            logger.info("No auto-analysis performed.")

        # Initialize the global app state with version tracking
        app_state = {
            "df": rent_roll_df,
            "issues": issues_list,
            "anthropic_client": Anthropic(api_key=anthropic_key),
            "openai_client": OpenAI(api_key=openai_key),
            "system_message": "",  # Will be populated below
            "df_versions": []  # Initialize empty version registry
        }

        # Save the initial version
        initial_version = save_dataframe_version(rent_roll_df, "Initial upload - original dataset")
        logger.info(f"Created initial dataframe version: {initial_version}")

        # Create system message with data understanding
        column_info = []
        for col in rent_roll_df.columns:
            try:
                dtype_str = str(rent_roll_df[col].dtype)
                column_info.append(f"- {col}: {dtype_str}")
            except Exception as e:
                column_info.append(f"- {col}: [Error determining type: {str(e)}]")
        column_info_str = "\n".join(column_info)
        # Calculate basic stats about the data
        data_stats = []
        for col in rent_roll_df.columns:
            try:
                if pd.api.types.is_numeric_dtype(rent_roll_df[col]):
                    stat = f"- {col}: min={rent_roll_df[col].min()}, max={rent_roll_df[col].max()}, mean={rent_roll_df[col].mean():.2f}, null={rent_roll_df[col].isna().sum()}"
                else:
                    unique_vals = rent_roll_df[col].nunique()
                    stat = f"- {col}: unique values={unique_vals}, null={rent_roll_df[col].isna().sum()}"
                data_stats.append(stat)
            except:
                data_stats.append(f"- {col}: [error calculating stats]")
        data_stats_str = "\n".join(data_stats)

        # Format issues for display
        issues_text = "\n".join([f"- {issue}" for issue in issues_list])

        system_message = f"""
        You are a Commercial Real Estate rent roll assistant that has analyzed a rent roll and found the following issues:

        {issues_text}

        The rent roll data has {len(rent_roll_df)} rows and {len(rent_roll_df.columns)} columns.

        Column information:
        {column_info_str}

        Data statistics:
        {data_stats_str}

        When helping the user, follow these critical guidelines:
        1. DO NOT generate placeholder code with fake column names. Work ONLY with the actual columns from the dataframe.
        2. NEVER assume column names that don't exist in the actual data.
        3. Always start by examining the first few rows to understand the meaning of each column.
        4. If you can't identify which columns contain certain information, clearly state this limitation.
        5. DO NOT proceed with analysis using made-up column names that don't exist in the data.

        The entire dataframe is available as 'df' in the execution environment.

        Important instructions for code and calculations:
        1. ALWAYS share your chain of thought reasoning in your responses. For each analysis:
          - Begin with "**Thinking through this step by step:**" in bold
          - Clearly explain your understanding of the request
          - Describe your approach to solving the problem
          - Outline the data exploration steps you'll take
          - Explain why you're choosing specific columns and methods
          - Discuss any challenges you anticipate with the data structure
          This chain of thought should be visible to the user in your chat responses.
        """

        # Save the system message to the app state
        app_state["system_message"] = system_message

        # Clean up the temporary file
        os.unlink(file_path)
        logger.info("Temporary file removed")

        # Generate a preview of the data and issues
        preview_html = f"""
        <h3>Rent Roll Preview</h3>
        <p>Successfully loaded rent roll with {len(rent_roll_df)} rows and {len(rent_roll_df.columns)} columns.</p>
        {rent_roll_df.head(5).fillna('').to_html(index=False)}

        <h3>Identified Issues</h3>
        <ol>
        """

        # Format each issue for the HTML preview
        for issue in issues_list:
            # If issue starts with a number (like "1. Issue"), strip the number
            if issue and issue[0].isdigit() and ". " in issue[:5]:
                issue = issue[issue.find(". ")+2:]
            preview_html += f"<li>{issue}</li>"

        preview_html += """
        </ol>
        <p>You can now start asking questions in the chat below!</p>
        <p><strong>Note:</strong> This application uses GPT-4 for decision making and text responses,
        and Claude AI specifically for code generation and execution.</p>
        """
        version_choices = get_version_choices()
        # Make the chat interface visible
        logger.info("Setup complete. Ready for chat interaction.")
        return (
            "Rent roll loaded successfully! You can now start chatting.",
            preview_html,
            gr.update(visible=True),  # chatbot visibility
            gr.update(choices=version_choices, value=version_choices[-1] if version_choices else None)  # version dropdown
        )


    # Also update the error return:
    except Exception as e:
        logger.error(f"Error during rent roll processing: {e}")
        logger.error(traceback.format_exc())
        if 'file_path' in locals() and os.path.exists(file_path):
            os.unlink(file_path)
            logger.info("Cleaned up temporary file after error")
        return f"Error: {str(e)}", None, gr.update(visible=False), gr.update(choices=[], value=None)

In [13]:
def load_latest_version_for_editing():
    """Load the most recent version of the dataframe for editing"""
    global app_state

    if app_state is None or app_state["df"] is None:
        return None, "No data loaded. Please upload a rent roll first."

    try:
        # Use the current dataframe (which is the latest)
        df = app_state["df"].copy()
        df = df.fillna('')
        # Get version info
        if app_state["df_versions"]:
            latest_version = app_state["df_versions"][-1]
            version_info = f"Loaded version: {latest_version['name']} - {latest_version['description']}"
        else:
            version_info = "Loaded current data (no versions saved yet)"

        logger.info(f"Loaded dataframe for editing: {df.shape}")
        return df, version_info
    except Exception as e:
        logger.error(f"Error loading data for editing: {e}")
        return None, f"Error loading data: {str(e)}"

def save_edited_dataframe(edited_df, description):
    """Save the edited dataframe as a new version"""
    global app_state

    if edited_df is None or edited_df.empty:
        return "No data to save", gr.update()

    try:
        # Convert the edited dataframe to proper pandas DataFrame if needed
        if not isinstance(edited_df, pd.DataFrame):
            edited_df = pd.DataFrame(edited_df)

        # Generate a meaningful description
        if not description:
            description = "Manual edits via data editor"

        # Save as new version
        version_name = save_dataframe_version(edited_df, description)

        # Update the app state with the edited dataframe
        app_state["df"] = edited_df

        # Log the changes
        logger.info(f"Saved edited dataframe as version {version_name}")

        # Return success message and update the view
        return f"✅ Successfully saved as version {version_name}", gr.update(value=edited_df)

    except Exception as e:
        logger.error(f"Error saving edited dataframe: {e}")
        return f"❌ Error saving: {str(e)}", gr.update()

def load_specific_version(version_name):
    """Load a specific version for editing"""
    global app_state

    if not version_name:
        return None, "Please select a version to load"

    try:
        # Find the version file
        versions_dir = "rent_roll_versions"
        csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")

        if os.path.exists(csv_filename):
            df = pd.read_csv(csv_filename)
            df = df.fillna('')
            logger.info(f"Loaded version {version_name} for editing")
            return df, f"Loaded version: {version_name}"
        else:
            return None, f"Version file not found: {version_name}"

    except Exception as e:
        logger.error(f"Error loading version {version_name}: {e}")
        return None, f"Error loading version: {str(e)}"

def get_version_choices():
    """Get list of available versions for dropdown"""
    global app_state

    if app_state and "df_versions" in app_state and app_state["df_versions"]:
        choices = []
        for i, version in enumerate(app_state["df_versions"]):
            status = ""
            if i == 0:
                status = " (ORIGINAL)"
            elif i == len(app_state["df_versions"]) - 1:
                status = " (LATEST)"

            choices.append(f"{version['name']}{status}")
        return choices
    return []

def refresh_version_dropdown():
    """Refresh the version dropdown choices"""
    choices = get_version_choices()
    if choices:
        return gr.update(choices=choices, value=choices[-1])  # Default to latest
    return gr.update(choices=[], value=None)

In [14]:
# Global state for the application (Not part of graph state)
app_state = {
    "df": None,
    "anthropic_client": None,
    "openai_client": None,  # Added for GPT-4
    "issues": [],
    "system_message": ""
}

# Chat processing function using the agentic workflow
def chat(message, history):
    """Process user message through the agentic workflow and get response."""
    global app_state

    logger.info(f"Received chat message: {message[:50]}...")

    if app_state is None or app_state["df"] is None:
        logger.warning("Chat attempted before setup is complete")
        return history + [(message, "Please upload a rent roll file and set up your API keys first.")]

    # Get previous messages from history
    prev_messages = []
    if history:
        for user_msg, assistant_msg in history:
            prev_messages.append({"role": "user", "content": user_msg})
            prev_messages.append({"role": "assistant", "content": assistant_msg})

    # Create message list without system message
    all_messages = []
    all_messages.extend(prev_messages)

    # Add the current user message
    all_messages.append({"role": "user", "content": message})

    # Create a state dictionary for the graph
    state = {
        "messages": all_messages,
        "system_message": app_state["system_message"],  # Store separately
        "df": app_state["df"],
        "issues": app_state["issues"],
        "needs_clarification": False,
        "generate_code": False,
        "execution_plan": None,
        "clarification_question": None,
        "code_execution_results": None,
        "final_response": None,
        # Pass the API clients to the state
        "anthropic_client": app_state["anthropic_client"],
        "openai_client": app_state["openai_client"]
    }

    try:
        # Create the workflow if not already created
        if not hasattr(chat, "workflow"):
            chat.workflow = create_agentic_rent_roll_analyzer()
            logger.info("Created agentic workflow")

        # Run the workflow with the current state
        logger.info("Running agentic workflow")
        result = chat.workflow.invoke(state)

        # Get the final response from the result state
        final_response = result.get("final_response", "I'm sorry, I couldn't process your request.")
        logger.info(f"Received final response from workflow: {final_response[:50]}...")

        # Use the correct format for Gradio chatbot - must be a list of tuples (user_msg, bot_msg)
        history_list = list(history) if history else []
        history_list.append((message, final_response))

        logger.info("Chat response processing complete")
        return history_list

    except Exception as e:
        logger.error(f"Error processing chat: {e}")
        logger.error(traceback.format_exc())

        # Handle errors properly in the chat history format
        history_list = list(history) if history else []
        error_message = f"Error getting response: {str(e)}"
        history_list.append((message, error_message))
        return history_list

In [15]:
def view_data():
    """Return a preview of the rent roll data."""
    global app_state  # Use app_state instead of agent_state

    logger.info("View data requested")

    if app_state is None or app_state["df"] is None:  # Note the dictionary access with ["df"]
        logger.warning("View data requested but no data is loaded")
        return "No rent roll data loaded yet."

    # Generate HTML representation of the dataframe
    logger.info(f"Generating HTML preview of data with {len(app_state['df'])} rows")
    html = f"""
    <h3>Rent Roll Data</h3>
    <p>{len(app_state['df'])} rows × {len(app_state['df'].columns)} columns</p>
    {app_state['df'].head(10).fillna('').to_html(index=False)}
    """

    return html

In [16]:

def clear_chat():
    """Reset the chat history."""
    logger.info("Clearing chat history")
    return []  # Return empty list for Gradio chat history

In [17]:
def view_dataframe_versions():
    """Return HTML showing all versions of the rent roll dataframe."""
    global app_state
    logger.info("View dataframe versions requested")

    versions_dir = "rent_roll_versions"

    if not os.path.exists(versions_dir):
        logger.warning("No versions directory found")
        return "No version history found. Please save a version first."

    # Get all files in the versions directory
    try:
        all_files = os.listdir(versions_dir)
        # Match any CSV file containing rent_roll in the name
        version_files = [f for f in all_files if f.endswith('.csv') and 'rent_roll' in f]
    except Exception as e:
        logger.error(f"Error reading versions directory: {e}")
        return f"Error listing versions: {str(e)}"

    if not version_files:
        logger.warning("No version files found in directory")
        return f"No version files found in the versions directory ({versions_dir})."

    # Extract version information
    versions = []
    for file in version_files:
        # Extract the version name from the filename
        if file.startswith('rent_roll_v_'):
            version_name = file.replace('rent_roll_', '').replace('.csv', '')
        else:
            version_name = os.path.splitext(file)[0].replace('rent_roll_', '')

        # Get file stats
        try:
            file_path = os.path.join(versions_dir, file)
            file_stats = os.stat(file_path)
            file_size = file_stats.st_size
            modified_time = datetime.fromtimestamp(file_stats.st_mtime).strftime("%Y-%m-%d %H:%M:%S")

            # Try to get row and column counts
            df_info = ""
            try:
                temp_df = pd.read_csv(file_path)
                df_info = f"{len(temp_df)} rows × {len(temp_df.columns)} columns"
            except:
                df_info = "Unable to read file"

            # If we have version info in app_state
            description = ""
            is_original = False

            for v in app_state.get("df_versions", []):
                if v.get("name") == version_name:
                    description = v.get("description", "")
                    is_original = v.get("is_original", False)
                    break

            # If not found in app_state, use fallback description
            if not description and os.path.exists(file_path):
                description = "Found in directory"

            versions.append({
                'version_name': version_name,
                'file_size': file_size,
                'modified_time': modified_time,
                'df_info': df_info,
                'description': description,
                'is_original': is_original,
                'file_path': file_path
            })
        except Exception as e:
            logger.error(f"Error processing version file {file}: {e}")
            versions.append({
                'version_name': version_name,
                'file_size': 0,
                'modified_time': 'Error',
                'df_info': f"Error: {str(e)}",
                'description': '',
                'is_original': False,
                'file_path': os.path.join(versions_dir, file)
            })

    # Sort versions by modification time
    versions.sort(key=lambda x: x['modified_time'])

    # Create basic HTML table without zebra striping
    html = """
    <h3 style="color: white;">Rent Roll Dataframe Version History</h3>
    """

    html += f"""
    <p style="color: white;">Found {len(versions)} version(s) in {versions_dir}</p>
    <table border="1" cellpadding="5" cellspacing="0" style="width: 100%; border-collapse: collapse; color: white;">
        <thead style="background-color: #009879;">
            <tr>
                <th style="text-align: left; padding: 10px;">Version Name</th>
                <th style="text-align: left; padding: 10px;">Status</th>
                <th style="text-align: left; padding: 10px;">Created</th>
                <th style="text-align: left; padding: 10px;">Size</th>
                <th style="text-align: left; padding: 10px;">Data</th>
                <th style="text-align: left; padding: 10px;">Description</th>
            </tr>
        </thead>
        <tbody>
    """

    for i, v in enumerate(versions):
        # No alternating rows - all cells have the same background and text color
        # Always use dark background with white text for all rows

        # Determine status badge
        if i == 0 or v.get('is_original'):
            status_html = '<span style="background-color: #3949ab; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">ORIGINAL</span>'
        elif i == len(versions) - 1:
            status_html = '<span style="background-color: #43a047; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">LATEST</span>'
        else:
            # Middle version with orange badge
            status_html = f'<span style="background-color: #f57c00; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">v{i+1}</span>'

        # All rows have dark background and white text
        html += f"""
        <tr style="background-color: #25292e; color: white; border-bottom: 1px solid #333;">
            <td style="padding: 10px;"><code style="font-family: monospace; font-weight: bold;">{v['version_name']}</code></td>
            <td style="padding: 10px;">{status_html}</td>
            <td style="padding: 10px;">{v['modified_time']}</td>
            <td style="padding: 10px;">{round(v['file_size']/1024, 2)} KB</td>
            <td style="padding: 10px;">{v['df_info']}</td>
            <td style="padding: 10px;">{v['description']}</td>
        </tr>
        """

    html += """
        </tbody>
    </table>
    """

    logger.info(f"Generated version history display with {len(versions)} versions")
    return html

In [18]:
# Initialize the global agent state
agent_state = None
custom_css = """
.chatbot-container .message-wrap .message.bot pre {
    white-space: pre !important;
    overflow-x: auto !important;
    max-width: 100% !important;
}
.chatbot-container .message-wrap .message.bot code {
    white-space: pre !important;
}
"""
# Define the Gradio interface
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue"), css=custom_css) as demo:
    gr.Markdown("# Agentic Commercial Real Estate Rent Roll Analyzer")
    gr.Markdown("## Hybrid AI System: GPT-4 for Decision Making & Claude for Code Generation")

    with gr.Tab("Setup"):
        with gr.Row():
            with gr.Column():
                file_input = gr.File(label="Upload Rent Roll Excel File (.xlsx, .xls)")

                # Add separate API key inputs for OpenAI and Anthropic
                anthropic_api_key = gr.Textbox(
                    label="Anthropic API Key (Optional - for code generation)",
                    placeholder="Leave blank to use the default API key",
                    type="password"
                )

                openai_api_key = gr.Textbox(
                    label="OpenAI API Key (Optional - for decision making and text responses)",
                    placeholder="Leave blank to use the default API key",
                    type="password"
                )

                # Updated auto-analyze checkbox
                auto_analyze = gr.Checkbox(
                    label="Automatically analyze for issues using GPT-4",
                    value=True,
                    info="When checked, GPT-4 will automatically identify issues in your rent roll"
                )

                upload_button = gr.Button("Load Rent Roll & Start Chat", variant="primary")

            with gr.Column():
                result = gr.Textbox(label="Status")
                preview = gr.HTML(label="Data Preview")

    with gr.Tab("Chat"):
        # Changed button text to reflect version history functionality
        view_versions_btn = gr.Button("View Version History")
        data_view = gr.HTML()
        chatbot = gr.Chatbot(label="Agentic Rent Roll Analysis Chat", height=500, type="tuples")

        with gr.Row():
            with gr.Column(scale=4):
                msg = gr.Textbox(label="Your question", placeholder="Ask about the rent roll...", lines=2)
            with gr.Column(scale=1):
                send_btn = gr.Button("Send", variant="primary")

        clear_btn = gr.Button("Clear Chat History")

        # Set up event handlers with proper return values for Gradio chatbot
        msg.submit(
            chat,
            inputs=[msg, chatbot],
            outputs=[chatbot]
        ).then(
            lambda: "", None, msg  # Clear the message box after sending
        )

        send_btn.click(
            chat,
            inputs=[msg, chatbot],
            outputs=[chatbot]
        ).then(
            lambda: "", None, msg  # Clear the message box after sending
        )

        clear_btn.click(clear_chat, None, chatbot)
        # Changed to use the new version history function
        view_versions_btn.click(view_dataframe_versions, None, data_view)

    with gr.Tab("Edit Data"):
        gr.Markdown("""
        ### 📝 Edit Rent Roll Data

        You can directly edit cells in the table below, just like in Excel.
        - Click on any cell to edit it
        - Use Tab or arrow keys to navigate
        - Changes are not saved until you click "Save as New Version"
        """)

        with gr.Row():
            with gr.Column(scale=3):
                # Version selector
                version_dropdown = gr.Dropdown(
                    label="Select Version to Edit",
                    choices=get_version_choices(),
                    value=None,
                    interactive=True
                )

            with gr.Column(scale=1):
                refresh_versions_btn = gr.Button("🔄 Refresh Versions", size="sm")
                load_version_btn = gr.Button("📂 Load Selected Version", variant="primary")

        # Status display
        edit_status = gr.Textbox(label="Status", interactive=False)

        # The editable dataframe
        editable_df = gr.Dataframe(
            label="Editable Data (Click any cell to edit)",
            interactive=True,  # This makes it editable!
            wrap=True,
            max_height=500,  # Changed from height to max_height
            column_widths=["100px"] * 20,  # Adjust based on your needs
        )

        # Save controls
        with gr.Row():
            with gr.Column(scale=3):
                save_description = gr.Textbox(
                    label="Description of Changes",
                    placeholder="e.g., 'Updated rent for units 101-105' or 'Corrected typos in tenant names'",
                    lines=2
                )

            with gr.Column(scale=1):
                save_changes_btn = gr.Button("💾 Save as New Version", variant="primary", size="lg")

        save_status = gr.Textbox(label="Save Status", interactive=False)

        # Quick actions section
        with gr.Accordion("Quick Actions", open=False):
            gr.Markdown("""
            ### Bulk Operations
            Use these buttons for common bulk edits:
            """)

            with gr.Row():
                # Add quick action buttons here in future
                gr.Button("🧹 Clean Empty Rows", size="sm", interactive=False)
                gr.Button("💵 Round All Currency", size="sm", interactive=False)
                gr.Button("📅 Fix Date Formats", size="sm", interactive=False)
                gr.Button("🔢 Recalculate Totals", size="sm", interactive=False)

        # Instructions
        gr.Markdown("""
        ---
        ### 💡 How to Use:
        1. **Load Data**: Click "Load Selected Version" to load a version for editing
        2. **Edit Cells**: Click on any cell and type to edit (just like Excel!)
        3. **Navigate**: Use Tab, Enter, or arrow keys to move between cells
        4. **Save Changes**: Enter a description and click "Save as New Version"

        ### ⚠️ Important Notes:
        - Changes are NOT auto-saved - you must click "Save as New Version"
        - Each save creates a new version, preserving the original
        - Large datasets may be slow to edit - consider using filters
        - To undo changes before saving, simply reload the version
        """)

        # Event handlers for Edit Data tab
        refresh_versions_btn.click(
            refresh_version_dropdown,
            outputs=[version_dropdown]
        )

        load_version_btn.click(
            lambda version: load_specific_version(version.split(" (")[0]) if version else (None, "No version selected"),
            inputs=[version_dropdown],
            outputs=[editable_df, edit_status]
        )

        save_changes_btn.click(
            save_edited_dataframe,
            inputs=[editable_df, save_description],
            outputs=[save_status, editable_df]
        ).then(
            refresh_version_dropdown,  # Refresh the dropdown after saving
            outputs=[version_dropdown]
        )

    # Initially hide the chat interface
    chatbot.visible = False

    # Updated upload button event with both API keys and version dropdown
    upload_button.click(
        upload_rent_roll,
        inputs=[file_input, anthropic_api_key, openai_api_key, auto_analyze],
        outputs=[result, preview, chatbot, version_dropdown]  # Added version_dropdown
    )

    # Updated style and help info
    gr.Markdown("""
    ## How to use this Agentic Rent Roll Analyzer:

    1. In the **Setup** tab, upload your rent roll Excel file
    2. Default API keys are already set, but you can provide your own if needed:
       - OpenAI API key is used for decision making and text responses
       - Anthropic API key is used for code generation and execution
    3. Choose whether to automatically analyze issues (recommended)
    4. Click "Load Rent Roll & Start Chat" to proceed
    5. Switch to the **Chat** tab to start asking questions about your rent roll
    6. Use the **Edit Data** tab to make direct edits to your data (like Excel)
    7. Use the "View Version History" button to see all saved versions of your dataframe

    ### Key Features:

    - **Hybrid AI approach**:
      - GPT-4 handles decision making and text responses
      - Claude AI specialized for code generation and execution

    - **Intelligent response mode**: The system automatically decides whether to:
      - Ask clarifying questions when your query is ambiguous
      - Provide simple explanations for straightforward questions
      - Generate and execute code for complex analytical questions

    - **Direct cell editing** (NEW):
      - Edit cells directly like in Excel
      - Load any version for editing
      - Save changes as new versions
      - Complete version history maintained

    - **Version tracking**: The system maintains a history of all dataframe versions
      - Original version is always the first one saved
      - Latest version is the most recent one saved
      - Specific versions can be referenced by name
      - All versions saved in both CSV and Excel formats

    - **Adaptive analysis**: Get insights tailored to your specific needs without unnecessary complexity

    - **Conversational interface**: Chat naturally about your rent roll data

    The assistant can analyze your rent roll data, identify issues, and suggest solutions - all in a natural conversational format.
    """)

<ipython-input-18-e4d2627bcabc>:53: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Agentic Rent Roll Analysis Chat", height=500, type="tuples")


In [ ]:
# Run the application
if __name__ == "__main__":
    logger.info("Starting Agentic Rent Roll Analyzer application")
    demo.launch(debug=True)
    logger.info("Application shutdown")

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://152cfbb8559a528965.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✓ Saved dataframe version v_20250525_012804: Initial upload - original dataset
  - CSV: rent_roll_versions/rent_roll_v_20250525_012804.csv
  - Excel: rent_roll_versions/rent_roll_v_20250525_012804.xlsx


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1965: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  state[block._id] = block.__class__(**kwargs)


In [20]:
!rm -rf /content/rent_roll_versions_older_versions2